# Mujoco RL template
Mujocoを用いた強化学習のために環境を簡単につくるためのひな型

##  参考
- [Gymnasium Documentation](https://gymnasium.farama.org )
- [test_mujoco_custom_env.py](https://github.com/Farama-Foundation/Gymnasium/blob/main/tests/envs/mujoco/test_mujoco_custom_env.py)
- [GitHub mujoco_py_env.py](https://github.com/Farama-Foundation/Gymnasium/blob/main/gymnasium/envs/mujoco/mujoco_py_env.py#L108)
- [GitHub inverted_pendulum.py](https://github.com/Farama-Foundation/Gymnasium/blob/main/gymnasium/envs/mujoco/inverted_pendulum.py)
- [【Python】物理エンジンMuJoCoの紹介＆MJCFドキュメント【MuJoCoチュートリアル①】](https://qiita.com/Yayoi-Habami/items/1bf5a3e05b1516a90381)
- [mujoco_pyの使用例](https://qiita.com/mk_yng/items/3160b48f32144f352b05#site%E3%82%BF%E3%82%B0%E3%81%AE%E3%81%A4%E3%81%84%E3%81%9F%E3%83%91%E3%83%BC%E3%83%84%E3%81%AE%E3%82%BF%E3%82%B9%E3%82%AF%E7%A9%BA%E9%96%93%E4%B8%AD%E3%81%AE%E4%BD%8D%E7%BD%AE%E3%81%A8%E9%80%9F%E5%BA%A6%E3%82%92%E5%8F%96%E5%BE%97%E3%81%99%E3%82%8B)

# Settings

In [ ]:
ENV_NAME = "sampleEnv-v0"
NUM_EPISODES = 100  # 最大試行回数
MAX_STEPS = 1000  # 1試行のstep数
MODEL_PATH = "your/model/path/and/name.th"
XML_FILE = "./sample.xml"
TENSORBOARD_LOG = "./logs"  # tensorboardのログ保存先
FRAME_SKIP = 50  #制御周期に合わせて設定

In [ ]:
# import json
# import time
# import requests

# class SlackNotifyBot(object):
#     def __init__(self, access_token):
#         self.__headers = {'Authorization': 'Bearer ' + access_token}

#     def send_to_slack(self, message,slack_ch):
#         API_URL = "https://slack.com/api/chat.postMessage"
#         headers = self.__headers
#         data = {
#             'channel': slack_ch,
#             'text': message
#         }
#         r = requests.post(API_URL, headers=headers, data=data)

# class LINENotifyBot(object):
#     API_URL = 'https://notify-api.line.me/api/notify'

#     def __init__(self, access_token):
#         self.__headers = {'Authorization': 'Bearer ' + access_token}

#     def send_to_line(self, message, image=None, sticker_package_id=None, sticker_id=None):
#         payload = {
#             'message': message,
#             'stickerPackageId': sticker_package_id,
#             'stickerId': sticker_id,
#         }
#         files = {'imageFile': open(image, 'rb')} if image else {}
#         requests.post(
#             LINENotifyBot.API_URL,
#             headers=self.__headers,
#             data=payload,
#             files=files,
#         )

# with open("./settings.json", "r", encoding="utf-8") as f:
#     settings = json.load(f)
# line_token = settings["LINE_token"]["my_line_token"]
# slack_token = settings["slack_token"]["my_slack_token"]
# my_slack_channel = settings["slack_ch"]["my_slack_channel"]

# slack_bot = SlackNotifyBot(access_token=slack_token)
# line_bot = LINENotifyBot(access_token=line_token)

# Environment

In [ ]:
import os
import warnings

import numpy as np
import pytest
import random
import mujoco

import gymnasium
from gymnasium import utils
from gymnasium.envs.mujoco import MujocoEnv
from gymnasium.error import Error
from gymnasium.spaces import Box
from gymnasium.utils.env_checker import check_env


class MyEnv(MujocoEnv, utils.EzPickle):
    """
    A simple mujuco env to test third party mujoco env, using the `Gymansium.MujocoEnv` environment API.
    """

    metadata = {
        "render_modes": [
            "human",
            "rgb_array",
            "depth_array",
        ],
    }

    def __init__(self, xml_file=XML_FILE, frame_skip=FRAME_SKIP, **kwargs):

        utils.EzPickle.__init__(self, xml_file, frame_skip, **kwargs)

        MujocoEnv.__init__(
            self,
            xml_file,
            frame_skip=frame_skip,
            observation_space=None,  # needs to be defined after
            default_camera_config={},
            camera_name="tracking_camera", #XMLで定義したカメラの名前を指定
            **kwargs,
        )

        self.metadata = {
            "render_modes": [
                "human",
                "rgb_array",
                "depth_array",
            ],
            "render_fps": int(np.round(1.0 / self.dt)),
        }



        '''
        連続値をとる行動空間はenvs/mujoco/mujoco_env.pyにおける_set_action_spaceによって
        xmlで定義したアクチュエータから自動で行動空間を定義する．

        以下は離散の場合の行動空間の定義の例．
        '''
        self.action_space = gymnasium.spaces.Discrete(5)

        # 観測空間の定義
        obs_size = len(self.data.sensordata) #センサー数の取得
        self.observation_space = Box(
            low=-np.inf, high=np.inf, shape=(obs_size,), dtype=np.float64
        )

        self.tire_torque = 0.02 #タイヤのトルク，N/m



    def step(self, action):


        if action == 0:
            set_action = (self.tire_torque, self.tire_torque)
        elif action == 1:
            set_action = (-self.tire_torque, self.tire_torque)
        elif action == 2:
            set_action = (self.tire_torque, -self.tire_torque)
        elif action == 3:
            set_action = (self.tire_torque, self.tire_torque/2)
        elif action == 4:
            set_action = (self.tire_torque/2, self.tire_torque)
        else:
            set_action = (0, 0)


        position_before = self.data.geom('main_body').xpos # xmlで定義したボディの位置を取得．名前で指定する例．

        self.do_simulation(set_action, self.frame_skip) # mujoco上で行動を実行し，シミュレーションを進める．
        obs= self._get_obs()

        position_after = self.data.geom('main_body').xpos


        # X方向に進んだ距離に応じて報酬
        reward = position_after[0] - position_before[0]

        # 壁にぶつかったらペナルティ
        for i in range(6):
            if self.data.sensordata[5+i].copy() > 0:
                reward -= 0.05
                break

        # 通路通り抜けたら終了
        if position_after[0] > 1:
            terminated = True
            reward += 1
        else:
            terminated = False

        state = self.state_vector() # 状態ベクトルの取得，各剛体の位置，姿勢，速度，角速度が一括で取得させる．


        # 転倒したらエピソードを終了する場合は以下のようにする．
        cos_half_theta = state[3]
        quaternion_z = state[6]


        sin_half_theta = np.sqrt(1 - cos_half_theta**2)

        if sin_half_theta == 0:
            theta = 0
        else :
            theta = 2 * np.arctan2(quaternion_z / sin_half_theta, cos_half_theta)

        not_truncation = (
            np.isfinite(state).all() and state[2] <= 0.5 and abs(theta) <= 1 #rad
        )

        truncation = not not_truncation

        # infoには追加の情報を格納する．
        info = self._get_reset_info()

        if self.render_mode == "human":
            self.render()

        return obs, reward, terminated, truncation, info


    def _get_obs(self):
        obs =np.array(
                [
                    self.data.sensordata[0].copy(), #センサーの値を取得．XMLで定義した順番で取得される．
                    self.data.sensordata[1].copy(),
                    self.data.sensordata[2].copy(),
                    self.data.sensordata[3].copy(),
                    self.data.sensordata[4].copy()
                ]
            ).ravel()
        return obs


    def reset_model(self):

        '''
        example:

        self.set_state(
            self.init_qpos
            + self.np_random.uniform(low=-0.1, high=0.1, size=self.model.nq),
            self.init_qvel + self.np_random.standard_normal(self.model.nv) * 0.1,)

        set_stateでモデルのジョイント位置qposと速度qvelを設定.上記はランダムな位置と初速を与える場合．
        '''
        qpos = self.init_qpos
        qvel = self.init_qvel
        rad = random.uniform(0,2*np.pi)

        # ロボットの初期位置と姿勢をランダムに設定，XMLで定義された剛体ごとに位置と姿勢(4元数)の７つの値を持つ．
        # init_qposで各剛体の初期位置と姿勢を一括で取得できる．
        # 定義された順番でqosのindexが割り当てられる．

        qpos[1] = random.uniform(-1, 1) #robot_y
        qpos[3] = np.cos(rad/2)
        qpos[4] = 0
        qpos[5] = 0
        qpos[6] = np.sin(rad/2)

        self.set_state(qpos,qvel)

        return self._get_obs()

    def _get_reset_info(self):
        return {"works": True}


    def viewer_setup(self):
        """
        ビューアの設定
            1. ビューアが初期化されていることを確認
            2. ビューアのインスタンスを変数vに代入
            3. カメラがトラッキングする対象をモデルの先頭の剛体(インデックス0)に設定
                - これによりカメラがロボットの根元を中心に映すようになる
            4. カメラとトラッキング対象との距離を設定
                - モデル全体の大きさ(self.model.stat.extent)の半分の距離に設定
            5. カメラの注視点の高さを設定
                - 具体的な値は v.model.stat.center[2] に基づいて決められている
        """
        assert self.viewer is not None  # ビューアが初期化されていることを確認
        v = self.viewer
        v.cam.trackbodyid = 0  # カメラがトラッキングする対象を先頭の剛体に設定
        v.cam.distance = self.model.stat.extent * 0.5  # カメラと対象との距離を設定
        v.cam.lookat[2] = 0.12250000000000005  # カメラの注視点の高さを設定


CHECK_ENV_IGNORE_WARNINGS = [
    f"\x1b[33mWARN: {message}\x1b[0m"
    for message in [
        "A Box observation space minimum value is -infinity. This is probably too low.",
        "A Box observation space maximum value is infinity. This is probably too high.",
        "For Box action spaces, we recommend using a symmetric and normalized space (range=[-1, 1] or [0, 1]). See https://stable-baselines3.readthedocs.io/en/master/guide/rl_tips.html for more information.",
    ]
]


@pytest.mark.parametrize("frame_skip", [1, 2, 3, 4, 5])
def test_frame_skip(frame_skip):
    """verify that custom envs work with different `frame_skip` values"""
    env = MyEnv(frame_skip=frame_skip)

    # Test if env adheres to Gym API
    with warnings.catch_warnings(record=True) as w:
        check_env(env.unwrapped, skip_render_check=True)
        env.close()
    for warning in w:
        if warning.message.args[0] not in CHECK_ENV_IGNORE_WARNINGS:
            raise Error(f"Unexpected warning: {warning.message}")


def test_xml_file():
    """Verify that the loading of a custom XML file works"""
    relative_path = "./tests/envs/mujoco/assets/walker2d_v5_uneven_feet.xml"
    env = MyEnv(xml_file=relative_path).unwrapped
    assert isinstance(env, MujocoEnv)
    assert env.data.qpos.size == 9

    full_path = os.getcwd() + "/tests/envs/mujoco/assets/walker2d_v5_uneven_feet.xml"
    env = MyEnv(xml_file=full_path).unwrapped
    assert isinstance(env, MujocoEnv)
    assert env.data.qpos.size == 9
    

    # note can not test user home path (with '~') because github CI does not have a home folder


def test_reset_info():
    """Verify that the environment returns info at `reset()`"""
    env = MyEnv()

    _, info = env.reset()
    assert info["works"] is True

In [ ]:
import gymnasium as gym
gym.envs.registration.register(id=ENV_NAME,entry_point=MyEnv)

# Learning

ランダムな行動

In [ ]:
import gymnasium as gym
from gymnasium.wrappers import RecordVideo
from gymnasium.wrappers import TimeLimit
import numpy as np
from tqdm import trange

env = gym.make(ENV_NAME, render_mode="human") # 環境の作成．render_modeを指定することで，ビデオの保存や画面表示を行うことができる．"human"は画面表示，"rgb_array"はビデオ保存用の画像データを返す．

# env = TimeLimit(env, max_episode_steps=MAX_STEPS) # wrappersを使って最大ステップ数を設定することもできる．
# env = RecordVideo(env, video_folder="./videos/")　# さらにビデオを保存することもできる．render_modeを'rgb_array'に設定すること

print("action_space:",env.action_space.shape)
print("observation_space:",env.observation_space.shape[0])

max_number_of_steps =  1000
num_consecutive_iterations = 100
num_episodes = 5
last_time_steps = np.zeros(num_consecutive_iterations)

for episode in range(num_episodes):
    # 環境の初期化
    observation = env.reset()
    episode_reward = 0

    for step in range(max_number_of_steps):
        #action = env.action_space.sample() # 連続値の場合はこのように行動をサンプリングすることもできる．
        action = episode

        # 行動の実行とフィードバックの取得
        observation, reward, done, _, info = env.step(action)
        # print("obs",observation)
        episode_reward += reward


        if done or step == max_number_of_steps - 1:
            print("sum_reward:",episode_reward)

            # print('%d Episode finished after %d time steps / mean %f' % (episode, t + 1,
            #     last_time_steps.mean()))
            # last_time_steps = np.hstack((last_time_steps[1:], [episode_reward]))
            break
env.close()

In [ ]:
# line_bot.send_to_line("Learning & evaluation complete")
# slack_bot.send_to_slack("Learning & evaluation complete",debug_channel)